# 1. Introducción
En este cuaderno se presenta un sencillo ejemplo para mostrar la diferencia de performance en la ejecución de un algoritmo trabajando de manera secuencial y paralela. Para esto se hace uso de hilos de CUDA para trabajar con ellos en paralelo.

La aplicación busca un valor dentro de un array de una sola dimensión. Se utiliza el algoritmo de búsqueda más básico: recorrer la estructura y validar si el valor buscado es el valor actual.


# 2. Armado del ambiente


In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 7.3MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=60b9cee9d33660be0629fb799473589c610dc9c348e99672e29a3a8fa7854268
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=06fdefa58c64f5956aafafcd509cdddea34399cfc0bc490ed26d19663645ec7c
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [ ]:
from datetime import datetime

import numpy

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# 3. Desarrollo
Ejecución de código en CPU.

In [ ]:
#@title 3.1 Parámetros de ejecución { vertical-output: true }

N = 500000#@param { type: 'number' }

tiempo_total_cpu = datetime.now()

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

array_cpu = numpy.arange(1, N + 1)
numpy.random.shuffle(array_cpu)
buscado = numpy.random.randint(1, N + 1)
print(array_cpu)
print(buscado)

tiempo_bucle = datetime.now()

for idx in range(0, N):
  if array_cpu[idx] == buscado:
    indiceBuscado = idx
    break

tiempo_bucle = datetime.now() - tiempo_bucle

print("Tiempo bucle: ", tiempo_en_ms( tiempo_bucle ), "[ms]" )
print(indiceBuscado)

[ 21171  71215 157030 ... 215996  46966 355432]
445282
Tiempo bucle:  119.572 [ms]
309168


Ejecución de código en GPU usando CUDA

In [ ]:
#@title 3.1 Parámetros de ejecución { vertical-output: true }

N = 500000#@param { type: 'number' }

tiempo_total_gpu = datetime.now()

# CPU - Defino la memoria de los vectores en cpu.
array_cpu = numpy.arange(1, N + 1)
numpy.random.shuffle(array_cpu)
buscado = numpy.random.randint(1, N + 1)

# CPU - reservo la memoria GPU.
array_gpu = cuda.mem_alloc(array_cpu.nbytes)

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod(array_gpu, array_cpu)

module = SourceModule("""
__global__ void kernel_buscar(int buscado, int N, int *array_gpu)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if(array_gpu[idx] == buscado && idx < N)
  {
    return;
  }
}
""")

# CPU - Genero la función kernel.
kernel_fn = module.get_function("kernel_buscar")

tiempo_bucle_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo = 256
dim_bloque = numpy.int((N + dim_hilo - 1) / dim_hilo )
print("Thread x: ", dim_hilo, ", Bloque x:", dim_bloque)

kernel_fn(numpy.int32(buscado), numpy.int32(N), array_gpu, block = (dim_hilo, 1, 1), grid = (dim_bloque, 1, 1))

tiempo_bucle_gpu = datetime.now() - tiempo_bucle_gpu

print("Tiempo GPU: ", tiempo_en_ms(tiempo_bucle_gpu), "[ms]")



Thread x:  256 , Bloque x: 1954
Tiempo GPU:  0.38 [ms]


# Tabla de pasos

Pasos trabajando con CPU

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importación de módulos.
CPU      |  datetime.now()        | Toma el tiempo inicial.
CPU      |  numpy.arange(1, N + 1) | Crea el vector con números de 1 al N.
CPU      |  numpy.random.randint(1, N + 1) | Obtiene un valor aleatorio que se buscará
CPU      |  for...                | Recorre el vector buscando el valor.
CPU      |  datetime.now()        | Toma el tiempo final.
CPU      |  print()               | Informa los resultados.

Pasos trabajando con GPU

 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importación de módulos.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.arange(1, N + 1) | Crea el vector con números de 1 al N.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_hilo, dim_bloque  | Calcula las dimensiones.
**GPU**  |  kernel_fn()           | Ejecuta el kernel en GPU
CPU      |  print()               | Informo los resultados.

# Conclusiones

Se evidencia la diferencia de performance entre la primera y la segunda implementación. Los hilos de CUDA encuentran mucho más rápido el valor buscado.

Es probable que en arrays con un N más pequeño no sea tan evidente la diferencia, ya que mientras menos sea N más probabilidad existe de que el valor buscado se encuentre entre los primeros.

Se podría continuar con el ejercicio aplicando distintos algoritmos de búsqueda. A partir de un parámetro podría seleccionarse cual usar. Así podría compararse distintos algoritmos de manera secuencial y paralela.

# Bibliografía
Interactive Forms

https://colab.research.google.com/notebooks/forms.ipynb

An even easier introduction to CUDA

https://developer.nvidia.com/blog/even-easier-introduction-cuda/